# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import gmaps.datasets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [57]:
weather_data = pd.read_csv('../WeatherPy/output_data/cities.csv')
weather_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nome,64.5011,-165.4064,14.00,73,90,17.27,US,1610423337
1,1,port augusta,-32.5000,137.7667,95.00,43,0,17.27,AU,1610423217
2,2,sucua,-2.4667,-78.1667,67.32,79,84,2.66,EC,1610423337
3,3,thompson,55.7435,-97.8558,19.40,93,75,4.61,CA,1610423337
4,4,namibe,-15.1961,12.1522,70.70,68,67,5.73,AO,1610423337
...,...,...,...,...,...,...,...,...,...,...
545,545,isangel,-19.5500,169.2667,77.00,100,90,7.14,VU,1610423391
546,546,sao jose da coroa grande,-8.8978,-35.1478,76.91,81,99,7.00,BR,1610423391
547,547,cabedelo,-6.9811,-34.8339,77.00,88,0,2.30,BR,1610423239
548,548,marrakesh,31.6315,-8.0083,39.20,93,40,2.30,MA,1610423160


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [58]:
#Access maps with unique API key
gmaps.configure(api_key = g_key)

In [59]:
#Store latitude and longitude in locations
locations = weather_data[['Lat', 'Lng']]

humidity = weather_data["Humidity"].astype(float)

In [60]:
#Plot Heatmap
fig = gmaps.figure(center = (35.0, 0.0), zoom_level=1.5)
max_humidity = np.max(humidity)

#Create heat layer
#heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 2, opacity = 0.5)
#fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [61]:
#Narrow the dataframe
narrowed_city_df = weather_data.loc[(weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80) & (weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,mataura,-46.1927,168.8643,77.68,34,0,0.31,NZ,1610423339
105,105,dangriga,16.9692,-88.2321,75.00,92,0,1.99,BZ,1610423347
184,184,richards bay,-28.7830,32.0377,72.86,88,0,3.04,ZA,1610423354
215,215,tura,25.5198,90.2201,72.00,43,0,4.16,IN,1610423357
256,256,ambovombe,-25.1667,46.0833,77.76,70,0,7.63,MG,1610423362
281,281,pisco,-13.7000,-76.2167,70.00,88,0,9.22,PE,1610423065
319,319,sakaraha,-22.9000,44.5333,76.86,69,0,3.83,MG,1610423367
337,337,sembe,1.6481,14.5806,71.49,83,0,3.36,CG,1610423369
381,381,caravelas,-17.7125,-39.2481,74.86,88,0,6.15,BR,1610423374
391,391,ipixuna,-1.7625,-48.8043,77.00,94,0,2.30,BR,1610423375


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [62]:
#Create a hotel dataframe
hotel_df = pd.DataFrame(narrowed_city_df).reset_index()
hotel_df = hotel_df.drop(hotel_df.columns[0], axis=1)

#Add a "Hotel Name" column
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,21,mataura,-46.1927,168.8643,77.68,34,0,0.31,NZ,1610423339,
1,105,dangriga,16.9692,-88.2321,75.00,92,0,1.99,BZ,1610423347,
2,184,richards bay,-28.7830,32.0377,72.86,88,0,3.04,ZA,1610423354,
3,215,tura,25.5198,90.2201,72.00,43,0,4.16,IN,1610423357,
4,256,ambovombe,-25.1667,46.0833,77.76,70,0,7.63,MG,1610423362,
5,281,pisco,-13.7000,-76.2167,70.00,88,0,9.22,PE,1610423065,
6,319,sakaraha,-22.9000,44.5333,76.86,69,0,3.83,MG,1610423367,
7,337,sembe,1.6481,14.5806,71.49,83,0,3.36,CG,1610423369,
8,381,caravelas,-17.7125,-39.2481,74.86,88,0,6.15,BR,1610423374,
9,391,ipixuna,-1.7625,-48.8043,77.00,94,0,2.30,BR,1610423375,


In [63]:
#Search for hotels with 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set parameters to search
params = {
         "type" : "hotel",
         "keyword" : "hotel",
         "radius" : 5000,
         "key" : g_key
}

In [64]:
count = 0

for index, row in hotel_df.iterrows():
    
    #Get city name from the list
    city_name = row["City"]
    lat = row['Lat']
    lng = row['Lng']
    
    #add params
    params['location'] = f'{lat},{lng}'
    
    count += 1
    
    print(f"Retrieving Results for: {city_name}.")
    
    #Make request
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        print(f"Hotel Found")
        print("------------")
        
    except:
        hotel_df.loc[index, "Hotel Name"] = "No Hotel in Range"
        print("Missing field/result... skipping.")
        print("------------")
        
        #set sleep time to avoid error
        time.sleep(1.00)
        
#Once search complete
print("Searching Complete")

Retrieving Results for: mataura.
Hotel Found
------------
Retrieving Results for: dangriga.
Hotel Found
------------
Retrieving Results for: richards bay.
Hotel Found
------------
Retrieving Results for: tura.
Hotel Found
------------
Retrieving Results for: ambovombe.
Hotel Found
------------
Retrieving Results for: pisco.
Hotel Found
------------
Retrieving Results for: sakaraha.
Hotel Found
------------
Retrieving Results for: sembe.
Missing field/result... skipping.
------------
Retrieving Results for: caravelas.
Hotel Found
------------
Retrieving Results for: ipixuna.
Missing field/result... skipping.
------------
Retrieving Results for: beloha.
Missing field/result... skipping.
------------
Retrieving Results for: cabedelo.
Hotel Found
------------
Searching Complete


In [65]:
#Store hotel results
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,21,mataura,-46.1927,168.8643,77.68,34,0,0.31,NZ,1610423339,Falls Hotel
1,105,dangriga,16.9692,-88.2321,75.00,92,0,1.99,BZ,1610423347,Pal's On the Beach
2,184,richards bay,-28.7830,32.0377,72.86,88,0,3.04,ZA,1610423354,BON Hotel Waterfront Richards Bay
3,215,tura,25.5198,90.2201,72.00,43,0,4.16,IN,1610423357,Hotel Polo Orchid
4,256,ambovombe,-25.1667,46.0833,77.76,70,0,7.63,MG,1610423362,Hotel Le Source
5,281,pisco,-13.7000,-76.2167,70.00,88,0,9.22,PE,1610423065,HOSTAL MIRAMAR
6,319,sakaraha,-22.9000,44.5333,76.86,69,0,3.83,MG,1610423367,Hôtel 2000
7,337,sembe,1.6481,14.5806,71.49,83,0,3.36,CG,1610423369,No Hotel in Range
8,381,caravelas,-17.7125,-39.2481,74.86,88,0,6.15,BR,1610423374,Mar Doce de Abrolhos
9,391,ipixuna,-1.7625,-48.8043,77.00,94,0,2.30,BR,1610423375,No Hotel in Range


In [66]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [67]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))